In [2]:
fang = pd.concat([pd.read_csv(f) for f in glob.glob(os.getcwd()+ '/data.tmp/raw/fang/'+'*.csv')], ignore_index = True)
fang.to_csv(os.getcwd()+"/data.tmp/processed/fang.csv",index=False)
fang

NameError: name 'pd' is not defined

End of loading in csv files

# Stock prediction using LSTM
In this notebook, we have attempted to use LSTM to predict the closing price of FANG stocks. First, we need to include some modules that will be used for the plotting and ML models.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
plt.style.use("seaborn-pastel")
%matplotlib inline

from pandas_datareader.data import DataReader
from datetime import datetime


For this example, we will use Apple, Google, Microsoft, Amazon, Facebook, and Netflix. We can use DataReader to get live data from Yahoo on these stocks.

In [ ]:
companies = ['AAPL', 'GOOG', 'MSFT', 'AMZN', 'FB', 'NFLX']
end = datetime.now()
start = datetime(end.year - 1, end.month, end.day)

for ticker in companies:
    globals()[ticker] = DataReader(ticker, 'yahoo', start, end)

We can start by doing analysis on one of the stocks, Amazon. Lets get the stock prices for the past 8 years.

In [ ]:
df = DataReader('AMZN', data_source='yahoo', start='2012-01-01', end=datetime.now())
df

We have information on the high, low, open, close, and volume for each day in the past year. If we look at the plot of the close price (which is what we'll use for analysis), you can see the fluctiation in price.

In [3]:
plt.figure(figsize=(16,8))
plt.title('Close Price History')
plt.plot(df['Close'])
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.show()

NameError: name 'plt' is not defined

for our dataset, we will just be working with the Close prices. For this example, we can use 60% of the data for training, and 40% for testing.

In [ ]:
data = df.filter(['Close'])
dataset = data.values
training_data_len = int(np.ceil( len(dataset) * .6 ))

training_data_len

We need to scale the data so that it is easier to work with in the model.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

Next, let's grab some training data for the model.

In [ ]:

train_data = scaled_data[0:int(training_data_len), :]
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
        
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


Keras makes it easy to implement LSTM.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences= False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(x_train, y_train, batch_size=1, epochs=1)

Finally, we can test the model on the 40% left of the data, and calculate the RMSE.

In [ ]:

test_data = scaled_data[training_data_len - 60: , :]
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
    
x_test = np.array(x_test)

x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
rmse

As you can see, our model did pretty well. On average, it predicted a bit low on the closing prices.

In [ ]:
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='upper left')
plt.show()

In [ ]:
valid

Now let's use the method we just developed again, but on the remaining tickers.

In [ ]:
for i, company in enumerate(companies, 1):
    df = DataReader(company, data_source='yahoo', start='2012-01-01', end=datetime.now())
    data = df.filter(['Close'])
    dataset = data.values
    training_data_len = int(np.ceil( len(dataset) * .6 ))

    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)


    train_data = scaled_data[0:int(training_data_len), :]
    x_train = []
    y_train = []

    for i in range(60, len(train_data)):
        x_train.append(train_data[i-60:i, 0])
        y_train.append(train_data[i, 0])
            
    x_train, y_train = np.array(x_train), np.array(y_train)

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape= (x_train.shape[1], 1)))
    model.add(LSTM(50, return_sequences= False))
    model.add(Dense(25))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(x_train, y_train, batch_size=1, epochs=1)
    test_data = scaled_data[training_data_len - 60: , :]
    x_test = []
    y_test = dataset[training_data_len:, :]
    for i in range(60, len(test_data)):
        x_test.append(test_data[i-60:i, 0])
        
    x_test = np.array(x_test)

    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)

    rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
    train = data[:training_data_len]
    valid = data[training_data_len:]
    valid['Predictions'] = predictions
    plt.figure(figsize=(16,8))
    plt.title(company)
    plt.xlabel('Date', fontsize=18)
    plt.ylabel('Close Price USD ($)', fontsize=18)
    plt.plot(train['Close'])
    plt.plot(valid[['Close', 'Predictions']])
    plt.legend(['Train', 'Val', 'Predictions'], loc='upper left')
    plt.show()

End of LSTM

In [ ]:
import os
import pandas as pd
import numpy as np

# models
from fbprophet import Prophet

# plots
import matplotlib.pyplot as plt
import seaborn as sns

color = sns.color_palette()
sns.set_style('darkgrid')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_faang = pd.read_csv(os.getcwd() +"/data.tmp/processed/fang.csv", low_memory=False)
df_faang.head()

In [ ]:
df_faang.isnull().sum()

There is no empty data. No rows needs to be dropped

In [ ]:
df_faang.info()

We need to make change the Date column to Date type and make it to be the index

# Data Cleaning

In [ ]:
df_faang["Date"] = pd.to_datetime(df_faang["Date"])
df_faang = df_faang.set_index("Date")
df_faang.head()

# Seperate the FAANG dataframe 

In [ ]:
df_facebook = df_faang.loc[df_faang['Name'] == 'FB']
df_apple = df_faang.loc[df_faang['Name'] == 'AAPL']
df_amazon = df_faang.loc[df_faang['Name'] == 'AMZN']
df_netflix = df_faang.loc[df_faang['Name'] == 'NFLX']
df_google = df_faang.loc[df_faang['Name'] == 'GOOGL']

list_faang_df = [
    (df_facebook, 'FB'),
    (df_apple, 'AAPL'),
    (df_amazon, 'AMZN'),
    (df_netflix, 'NFLX'),
    (df_google, 'GOOGL')]

In [ ]:
df_google.head()

# Understand more about the data with visualisation

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(df_google['Close'])
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("Google Stock Price")
plt.show()

Let's explore further

## Moving average
- Moving average is an indicator that smooth the volatility of daily price changes. 
- We will be using simple moving average which is the arithmetic mean over a specfic time period

We will use 50 days and 200 days moving average.

In [ ]:
df_google["MA50"] = df_google['Close'].rolling(window=50).mean()
df_google["MA200"] = df_google['Close'].rolling(window=200).mean()
df_google['ewma'] = df_google['Close'].ewm(halflife=0.5, min_periods=20).mean()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(df_google['MA50'], 'g--', label="Moving Average 50")
plt.plot(df_google['MA200'], 'r--', label="Moving Average 200")
plt.plot(df_google['Close'], label="Close")
plt.title("Moving average for GOOGL")
plt.legend()
plt.show()

## Let's take the last 200 trading days for Moving Average

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(df_google['MA50'].iloc[-200:], 'g--', label="Moving Average 50")
plt.plot(df_google['MA200'].iloc[-200:], 'r--', label="Moving Average 200")
plt.plot(df_google['Close'].iloc[-200:], label="Close")
plt.title("GOOGL Last 200 trading days for Moving average")
plt.legend()
plt.show()

Moving Average 50 days intersect with Moving Average 200 days. If the MA50 cross above MA200, it is an indication to buy. Otherwise, sell
<br>
### Our data visualisation with moving average indicate that we should buy 

## Explore further with Bollinger Bands

1. Middle Band= 20-day simple moving average (SMA)
2. Upper Band= 20-day SMA+(20-day standard deviation of price x 2)
3. Lower Band= 20-day SMA–(20-day standard deviation of price x 2)

Bollinger Bands illustrate the relative strength or momentum of a stock

In [ ]:
df_google['middle_band'] = df_google['Close'].rolling(window=20).mean()
df_google['upper_band'] = df_google['Close'].rolling(window=20).mean() + df_google['Close'].rolling(window=20).std()*2
df_google['lower_band'] = df_google['Close'].rolling(window=20).mean() - df_google['Close'].rolling(window=20).std()*2

## Let's take the last 200 trading days for Bollinger Bands

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(df_google['upper_band'].iloc[-200:], 'g--', label="upper")
plt.plot(df_google['middle_band'].iloc[-200:], 'r--', label="middle")
plt.plot(df_google['lower_band'].iloc[-200:], 'y--', label="lower")
plt.plot(df_google['Close'].iloc[-200:], label="Close")
plt.title("GOOGL Last 200 trading days with Bollinger bands")
plt.legend()
plt.show()

### From bollinger bands, the trend of GOOGLE stock seems to be increasing 

***
# Use Prophet Model 
## Will GOOGLE stock increase in the future just like what we found out from the data visualization?

In [ ]:
df_prophet = df_google.copy()
df_prophet.reset_index(drop=False, inplace=True)
df_prophet = df_prophet[["Date","Close"]]
df_prophet.head()

According to Prophet documentation , we have to only have two column with ds and y columns

In [ ]:
df_prophet.rename(columns={
    "Date": "ds",
    "Close": "y"
}, inplace=True)
df_prophet.head()

In [ ]:
df_prophet.info()

Reduce the GOOGLE  stock rows to start from 2017-01-01 to the current date

In [ ]:
df_prophet = df_prophet.loc[df_prophet["ds"].dt.year >= 2017]
df_prophet.reset_index(drop=True, inplace=True)
df_prophet

In [ ]:
model_prophet = Prophet(daily_seasonality=True)
model_prophet.fit(df_prophet)

In [ ]:
future = model_prophet.make_future_dataframe(periods=365) 
prediction = model_prophet.predict(future)

In [ ]:
model_prophet.plot(prediction)
plt.title("Prediction of the GOOGLE Stock Price using the Prophet")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.show()

In [ ]:
model_prophet.plot_components(prediction)
plt.show()

Based on the estimated trends
- GOOGLE stock price is maximum mostly on Thursday
- Seasonality information indicates it is best to sell in january and buy in April

### Prophet predicts that GOOGLE stock price will increase in the upcoming year which matches up with our prediction from data visualisation

## Let's do the prediction for the rest of the ticker


In [ ]:
for df, ticker_name in list_faang_df[:-1]:
    
    df["MA50"] = df['Close'].rolling(window=50).mean()
    df["MA200"] = df['Close'].rolling(window=200).mean()
    df['ewma'] = df['Close'].ewm(halflife=0.5, min_periods=20).mean()

    plt.figure(figsize=(10,10))
    plt.plot(df['MA50'].iloc[-200:], 'g--', label="Moving Average 50")
    plt.plot(df['MA200'].iloc[-200:], 'r--', label="Moving Average 200")
    plt.plot(df['Close'].iloc[-200:], label="Close")
    plt.title(f"{ticker_name} Last 200 trading days for Moving average")
    
    plt.legend()
    plt.show()

    df['middle_band'] = df['Close'].rolling(window=20).mean()
    df['upper_band'] = df['Close'].rolling(window=20).mean() + df['Close'].rolling(window=20).std()*2
    df['lower_band'] = df['Close'].rolling(window=20).mean() - df['Close'].rolling(window=20).std()*2

    plt.figure(figsize=(10,10))
    plt.plot(df['upper_band'].iloc[-200:], 'g--', label="upper")
    plt.plot(df['middle_band'].iloc[-200:], 'r--', label="middle")
    plt.plot(df['lower_band'].iloc[-200:], 'y--', label="lower")
    plt.plot(df['Close'].iloc[-200:], label="Close")
    plt.title(f"{ticker_name} Last 200 trading days with Bollinger bands")
    
    plt.legend()
    plt.show()

    df_prophet = df.copy()
    df_prophet.reset_index(drop=False, inplace=True)
    df_prophet = df_prophet[["Date","Close"]]
    df_prophet.head()


    df_prophet.rename(columns={
        "Date": "ds",
        "Close": "y"
    }, inplace=True)


    df_prophet = df_prophet.loc[df_prophet["ds"].dt.year >= 2017]
    df_prophet.reset_index(drop=True, inplace=True)

    model_prophet = Prophet(daily_seasonality=True)
    model_prophet.fit(df_prophet)

    future = model_prophet.make_future_dataframe(periods=365) 
    prediction = model_prophet.predict(future)

    model_prophet.plot(prediction)
    plt.title(f"Prediction of the {ticker_name} Stock Price using the Prophet")
    plt.xlabel("Date")
    plt.ylabel("Close Price")
    plt.show()
    
    model_prophet.plot_components(prediction)
    plt.show()
    
    print("=" *90)

## Summary
Based on the estimated trends
- Stock price maximum mostly on:
    - Wednesday for Facebook, Amazon, Apple, Google
    - Thursday for Netflix
- Seasonality information:
    - Buy in April for Facebook, Apple, Amazon, Google
    - Buy in  December for Netflix
    - Sell in January for Facebook, Google
    - Sell in September for Apple, Amazon
    - Sell in July for Netflix

- All FAANG stock price will increase in upcoming future

End of Prophet

In [ ]:
import glob
import os
import datetime
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data_FAANG = pd.read_csv("data.tmp/processed/fang.csv")
data_FAANG["Date"] = pd.to_datetime(data_FAANG["Date"])
data_FAANG = data_FAANG.set_index("Date")

In [ ]:
df_facebook = data_FAANG.loc[df_faang["Name"] == "FB"]
df_apple = data_FAANG.loc[df_faang["Name"] == "AAPL"]
df_amazon = data_FAANG.loc[df_faang["Name"] == "AMZN"]
df_netflix = data_FAANG.loc[df_faang["Name"] == "NFLX"]
df_google = data_FAANG.loc[df_faang["Name"] == "GOOGL"]

 # Q1. Whether the Stock Market Is in An Uptrend
 
 Stocks go up because investors are optimistic about the future. This is shown on a stock chart in a number of ways. The most important and easiest way is to look for whether the bottoms on the stock chart are rising and if the market is in a general uptrend (moving from the lower left to the lower right). That means that the buyers are in control of the market, making the stock more likely to go higher than lower.

###  Facebook

In [ ]:
df_facebook_uptrend = ((df_facebook["High"] - df_facebook["Low"]) / df_facebook["Volume"]) * 100
df_apple_uptrend = ((df_apple["High"] - df_apple["Low"]) / df_apple["Volume"]) * 100
df_amazon_uptrend = ((df_amazon["High"] - df_amazon["Low"]) / df_amazon["Volume"]) * 100
df_netflix_uptrend = ((df_netflix["High"] - df_netflix["Low"]) / df_netflix["Volume"]) * 100
df_google_uptrend = ((df_google["High"] - df_google["Low"]) / df_google["Volume"]) * 100

In [ ]:
plt.figure(figsize=(16,8))
plt.title("The Stock Market Trend of Facebook")
plt.xlabel("Date", fontsize=15)
plt.ylabel("Market Trend", fontsize=15)
plt.plot(df_facebook_uptrend)
plt.show()

 Facebook's stock market is in an uptrend. As the graph shown above, it is rising and the market is generally in an up trend.

### Apple 

In [ ]:
plt.figure(figsize=(16,8))
plt.title("The Stock Market Trend of Apple")
plt.xlabel("Date", fontsize=15)
plt.ylabel("Market Trend", fontsize=15)
plt.plot(df_apple_uptrend)
plt.show()

Apple's stock market is in an extreme uptrend. As the graph shown above, it does not rise until 2010. It rises significantly after 2015.

### Amazon

In [ ]:
plt.figure(figsize=(16,8))
plt.title("The Stock Market Trend of Amazon")
plt.xlabel("Date", fontsize=15)
plt.ylabel("Market Trend", fontsize=15)
plt.plot(df_amazon_uptrend)
plt.show()

Amazon's stock market is also in an extreme uptrend. As the graph shown above, it does not rise until 2008. It rises significantly after 2016.

### Netflix

In [4]:
plt.figure(figsize=(16,8))
plt.title("The Stock Market Trend of Netflix")
plt.xlabel("Date", fontsize=15)
plt.ylabel("Market Trend", fontsize=15)
plt.plot(df_netflix_uptrend)
plt.show()

NameError: name 'plt' is not defined

Netfix's stock market is also in an extreme uptrend. As the graph shown above, it does not rise until 2014. It rises significantly after 2016.

### Google

In [ ]:
plt.figure(figsize=(16,8))
plt.title("The Stock Market Trend of Google")
plt.xlabel("Date", fontsize=15)
plt.ylabel("Market Trend", fontsize=15)
plt.plot(df_google_uptrend)
plt.show()

Google's stock market is in an uptrend. As the graph shown above, it rises stably and the market is generally in an up trend.

### In Conclusion

FAANG companies all perform pretty well in recent five years. All of their stock markets are in an uptrend obviously. 

# Q2. Is a Stock Worth Buying? 
  There are many ways to evaluate if a stock is worth buying. Historical Price is one of them. If the invester wants to invest a stock for the long term, he/ she needs to do more than look at a single company earnings report or current price performance.Looking at the historical price of last ten years or five years will give him/her a sense of whether a company can withstand tough stretches. However, the historical price are not a guarantee of future performance, but can at minimum be illustrative.
  
  To calculate the historical price of a stock, it requires the PE. ratio from the company so I would use simplified method to replace that complicated formula. 

### Facebook

In [ ]:
df_facebook_price = ((df_facebook["High"] - df_facebook["Low"]) / 2) * 100
df_apple_price = ((df_apple["High"] - df_apple["Low"]) / 2) * 100
df_amazon_price = ((df_amazon["High"] - df_amazon["Low"]) / 2) * 100
df_netflix_price = ((df_netflix["High"] - df_netflix["Low"]) / 2) * 100
df_google_price = ((df_google["High"] - df_google["Low"]) / 2) * 100


In [ ]:
plt.figure(figsize=(16,8))
plt.title("Historical Stock Price of Facebook")
plt.ylabel("Price", fontsize=15)
plt.plot(df_facebook_price)
plt.show()

From the long run, the historical price of Facebook's stock increases even though there are some ups and downs

### Apple 

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Historical Stock Price of Apple")
plt.ylabel("Price", fontsize=15)
plt.plot(df_apple_price)
plt.show()

After 2005, the historical price of Apple's stock increases significantly.

### Amazon 

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Historical Stock Price of Amazon")
plt.ylabel("Price", fontsize=15)
plt.plot(df_amazon_price)
plt.show()

From the long run, the historical price of Amazon's stock increases especially in the 2020.

### Netflix

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Historical Stock Price of Netflex")
plt.ylabel("Price", fontsize=15)
plt.plot(df_netflix_price)
plt.show()

From the long run, the historical price of Netflix's stock increases. Especially after 2016, it increases significantly.

### Google

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Historical Stock Price of Google")
plt.ylabel("Price", fontsize=15)
plt.plot(df_google_price)
plt.show()

In the long run, the historical price of Google's stock increases even though there are some ups and downs.

End of Ben's Questions

In [ ]:
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_faang = pd.read_csv(os.getcwd() +"/data.tmp/processed/fang.csv", low_memory=False)

df_faang["Date"] = pd.to_datetime(df_faang["Date"])
df_faang = df_faang.set_index("Date")

In [ ]:
df_facebook = df_faang.loc[df_faang['Name'] == 'FB']
df_apple = df_faang.loc[df_faang['Name'] == 'AAPL']
df_amazon = df_faang.loc[df_faang['Name'] == 'AMZN']
df_netflix = df_faang.loc[df_faang['Name'] == 'NFLX']
df_google = df_faang.loc[df_faang['Name'] == 'GOOGL']

## 1. Which date provided the highest percentage return/lost for each company?

In [ ]:
df_facebook_returns = ((df_facebook['Close'] - df_facebook['Open']) / df_facebook['Open']) * 100
df_apple_returns = ((df_apple['Close'] - df_apple['Open']) / df_apple['Open']) * 100
df_amazon_returns = ((df_amazon['Close'] - df_amazon['Open']) / df_amazon['Open']) * 100
df_netflix_returns = ((df_netflix['Close'] - df_netflix['Open']) / df_netflix['Open']) * 100
df_google_returns = ((df_google['Close'] - df_google['Open']) / df_google['Open']) * 100

### Facebook

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Facebook Percentage Returns/Lost')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Percentage Gain/Lost (%)', fontsize=18)
plt.plot(df_facebook_returns)
plt.show()

In [ ]:
print(df_facebook_returns[df_facebook_returns == df_facebook_returns.min()])
print(df_facebook_returns[df_facebook_returns == df_facebook_returns.max()])

On 18 May 2012, Facebook had its highest percentage decrease. They decreased in value by 9.084423%.

On 14 November 2012, Facebook had its highest percentage increase. They increased in value by 11.243782%.

### Apple

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Apple Percentage Returns/Lost')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Percentage Gain/Lost (%)', fontsize=18)
plt.plot(df_apple_returns)
plt.show()

In [ ]:
print(df_apple_returns[df_apple_returns == df_apple_returns.min()])
print(df_apple_returns[df_apple_returns == df_apple_returns.max()])

On 19 October 1987, Apple had its highest percentage decrease. They decreased in value by 24.352329%.

On 02 January 1998, Apple had its highest percentage increase. They increased in value by 19.266055%.

### Amazon

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Amazon Percentage Returns/Lost')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Percentage Gain/Lost (%)', fontsize=18)
plt.plot(df_amazon_returns)
plt.show()

In [ ]:
print(df_amazon_returns[df_amazon_returns == df_amazon_returns.min()])
print(df_amazon_returns[df_amazon_returns == df_amazon_returns.max()])

On 31 August 1998, Amazon had its highest percentage decrease. They decreased in value by 20.569056%.

On 03 January 2001, Amazon had its highest percentage increase. They increased in value by 28.899083%.

### Netflix

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Netflix Percentage Returns/Lost')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Percentage Gain/Lost (%)', fontsize=18)
plt.plot(df_netflix_returns)
plt.show()

In [ ]:
print(df_netflix_returns[df_netflix_returns == df_netflix_returns.min()])
print(df_netflix_returns[df_netflix_returns == df_netflix_returns.max()])

On 26 July 2002, Netlix had its highest percentage decrease. They decreased in value by 17.006801%.

On 10 October 2002, Netflix had its highest percentage increase. They increased in value by 34.02647%.

### Google

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Google Percentage Returns/Lost')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Percentage Gain/Lost (%)', fontsize=18)
plt.plot(df_google_returns)
plt.show()

In [ ]:
print(df_google_returns[df_google_returns == df_google_returns.min()])
print(df_google_returns[df_google_returns == df_google_returns.max()])

On 29 September 2008, Google had its highest percentage decrease. They decreased in value by 9.179758%.

On 28 October 2008, Google had its highest percentage increase. They increased in value by 8.759763%.

## 2. Are there any significance to these dates?

### Facebook
#### 18 May 2012, decreased in value by ~9%

18 May 2012 was Facebook's IPO, the day it made its debut to the stock market. It smashed the record for highest trading volume at that time with 570 million shares being traded. Facebook set its final IPO price at \$38. The first trade came in at \$42. However, it quickly took a reverse dropping down to \$38.23 by the end of the day leading to a final decrease by 9%. 

(Source: https://money.cnn.com/2012/05/18/technology/facebook-ipo-trading/index.htm)

#### 14 November 2012, Increased in value by ~11%

I was not able to find any significant event on 14 November 2012 relating to Facebook. They however did release their 3rd Quarter 2012 results on October 2012 with some great numbers which may have led to the increased in value.

Data Reported:
    * Monthly active users (MAUs) were 1.01 billion as of September 30, 2012, an increase of 26% year-over-year
    * Daily active users (DAUs) were 584 million on average for September 2012, an increase of 28% year-over-year
    * Mobile MAUs were 604 million as of September 30, 2012, an increase of 61% year-over-year 
(Source: https://investor.fb.com/investor-news/press-release-details/2012/Facebook-Reports-Third-Quarter-2012-Results/default.aspx)

### Apple
#### 19 October 1987, decreased in value by ~24%
Black Monday! On 19 October 1987 (also known as the Black Monday) the Dow Jones Industrial Average fell by 508 points ( a 22% decrease). According to Investopedia, Economists have attributed the crash to a combination of geopolitical events and the advent of computerized program trading that accelerated the selloff.

(Source: https://www.investopedia.com/terms/b/blackmonday.asp)

#### 02 January 1998, Increased in value by ~19%
There was not any significant event on 2 January 1998 relating to Apple. However on 10 Novemeber 1997, Apple Computer introduces the Power Macintosh G3 computer, released the PowberBook G3, and Steven Jobs announces further changes to Apple’s corporate strategy. Apple would now sell computers direct, both over the web and the phone, as Power Computing had done so well in the past. This may have attributed to the increase in Apple's value.

(Source: https://igotoffer.com/apple/history-apple-1997-1998)

### Amazon
#### 31 August 1998, decreased in value by ~21%
I was not able to find any significant event on 31 August 1998 relating to Amazon.

#### 03 January 2001, Increased in value by ~29%
There was not any significant event on 03 Januart 2001 but it is important to note that around the year 2000, the dot-com bubble burst destroyed many e-companies in the process, but Amazon survived and moved forward beyond the tech crash to become a huge player in online sales.

(Source: https://www.investopedia.com/terms/d/dotcom-bubble.asp)

### Netflix
#### 26 July 2002, decreased in value by ~17%
I was not able to find any significant event on 26 July 2002 relating to Netflix.

#### 10 October 2002, Increased in value by ~34%
I was not able to find any significant event on 10 October 2002 relating to Netflix.

These dates are however really close to Netflix's IPO date which is on 23 May 2002 which may (or may not) play a part on its volatility.

### Google
#### 29 September 2008, decreased in value by ~9%
On this day, the stock market crashed yet again with the Dow Jones Industrial Average losing nearly 778 points,after the House rejected the government's /$700 billion bank bailout plan.

(Source: https://money.cnn.com/2008/09/29/markets/markets_newyork/)


#### 28 October 2008, Increased in value by ~9/%
On this day, the Dow Jones industrial average added 889 points after having risen as much as 906 points earlier in the session. It was the Dow's second-biggest one-day point gain ever then, following a 936-point rally two weeks ago. The advance of 10.9/% was the sixth-biggest ever then.

(Source: https://money.cnn.com/2008/10/28/markets/markets_newyork/)

In [ ]:
End of Warren's Questions